In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import gzip
import lasagne
from lasagne import layers
from utility import *
import tensorflow as tf
%load_ext autoreload
%autoreload 2
%pylab inline

/home/bzr0014/anaconda2/envs/hicplus/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
/home/bzr0014/anaconda2/envs/hicplus/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Populating the interactive namespace from numpy and matplotlib


/home/bzr0014/.local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['colors', 'log']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
pylab.rcParams['figure.figsize'] = (15, 9)
sys.setrecursionlimit(10000)
conv2d1_filters_numbers = 8
conv2d1_filters_size = 9
conv2d2_filters_numbers = 8
conv2d2_filters_size = 1
conv2d3_filters_numbers = 1
conv2d3_filters_size = 5

down_sample_ratio = 16
learning_rate = 0.00001
epochs = 10
HiC_max_value = 100

In [3]:
low_resolution_samples = np.load(gzip.GzipFile('../data/GM12878_replicate_down16_chr19_22.npy.gz', "r")) * down_sample_ratio
high_resolution_samples = np.load(gzip.GzipFile('../data/GM12878_replicate_original_chr19_22.npy.gz', "r"))
N, d, w, h = low_resolution_samples.shape
low_resolution_samples = low_resolution_samples[:, 0, :, :].reshape(N, w, h, d)
high_resolution_samples = high_resolution_samples[:, 0, :, :].reshape(N, w, h, d)

In [4]:
print(low_resolution_samples.shape)
print(high_resolution_samples.shape)

(14689, 40, 40, 1)
(14689, 40, 40, 1)


In [5]:
sample_size = low_resolution_samples.shape[-2]
padding = conv2d1_filters_size + conv2d2_filters_size + conv2d3_filters_size - 3
half_padding = padding / 2
output_length = sample_size - padding

In [6]:
Y = high_resolution_samples[:, half_padding:(sample_size-half_padding), half_padding:(sample_size-half_padding), 0]
Y = Y.reshape(Y.shape[0], -2)
X = low_resolution_samples

In [7]:
print(X.shape)
print(Y.shape)

(14689, 40, 40, 1)
(14689, 784)


In [ ]:
output_model_name = '../model/test_model'

f = file(output_model_name + '.net', 'wb')

pickle.dump(net1,f,protocol=pickle.HIGHEST_PROTOCOL)
f.close()